# Lab 12 Web scrapping

In [0]:
from lxml import html
import requests

from bs4 import BeautifulSoup
import urllib3

## 1. Wyświetlenie źródła strony www

In [0]:
url = "https://boardgamegeek.com"
data = requests.get(url)

data.text

'\n<!DOCTYPE html>\n<html ng-app="GeekApp" lang="en-US" ng-cloak>\n<head>\n\t<meta charset=\'utf-8\'>\n\t<meta id="vp" name="viewport" content="width=device-width, initial-scale=1.0">\n\t\t<meta content=\'yes\' name=\'apple-mobile-web-app-capable\'>\n\t<meta content=\'IE=edge,chrome=1\' http-equiv=\'X-UA-Compatible\'>\n\n\t\t\t<title>BoardGameGeek | Gaming Unplugged Since 2000</title>\n\t\n\t\n<link rel="apple-touch-icon" \thref="https://cf.geekdo-static.com/icons/touch-icon180.png" />\n<link rel="shortcut icon" \t\thref="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />\n<link rel="icon" \t\t\t\t\thref="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />\n<link rel="search" \t\t\t\thref="/game-opensearch.xml" type="application/opensearchdescription+xml" title="BGG Game Search" />\n<meta name="apple-mobile-web-app-title" content="BGG">\n\n\n\t<meta property="og:image" content="https://cf.geekdo-static.com/images/opengraph/bgg_opengraph.png" />\n\n\n\

## 2. Parsowanie HTML

In [0]:
url = "https://boardgamegeek.com/"
data = requests.get(url)

page = html.fromstring(data.text)

In [0]:
page

<Element html at 0x7f866ffed098>

## 3. Pobieranie informacji o elemencie HTML

In [0]:
url = "https://boardgamegeek.com/"
data = requests.get(url)
page = html.fromstring(data.text)

xpath = '//*[@class="geekcentral_leftcol"]//h2//a'

foundElements = page.xpath(xpath)

for element in foundElements :
   print(element.tag, element.keys())
   for name, value in sorted(element.items()):
       print('%s = %r' % (name, value))

a ['href', 'class']
class = 'link-text stretched-link-secondary'
href = '{{thread.objecthref}}'
a ['class', 'ng-class', 'ng-href']
class = 'stretched-link link-text-color'
ng-class = "{'text-normal':data.config.showobjectlink}"
ng-href = '/thread/{{thread.threadid}}/{{thread.linkname}}'
a ['class', 'ng-href']
class = 'stretched-link link-text-color'
ng-href = '{{item.url}}'
a ['class', 'ng-href']
class = 'stretched-link link-text-color'
ng-href = '{{item.parent.href}}'
a ['class', 'ng-href']
class = 'stretched-link link-text-color'
ng-href = '/geekbay/link/{{item.itemid}}/{{item.auctionid}}'
a ['class', 'ng-href']
class = 'stretched-link link-text-color'
ng-href = '{{post.href}}'
a ['href', 'class']
class = 'link-text stretched-link-secondary'
href = '{{video.objecthref}}'
a ['class', 'ng-href']
class = 'stretched-link link-text-color text-normal'
ng-href = '{{video.href}}'
a ['class', 'ng-href']
class = 'stretched-link link-text-color'
ng-href = '{{list.href}}'
a ['href', 'class']
cla

## 4. Parsowanie tabel 

Na przykładzie wyświetlenia nagłówków tabeli

In [0]:
url = "https://boardgamegeek.com/browse/boardgame"
data = requests.get(url)

page = html.fromstring(data.text)
table_div = page.get_element_by_id('collection')

table = table_div.xpath('./*[@class="table-responsive"]/table')[0]

headers = [label for label in table.xpath('.//th')]

labels = []

for header in headers:
    anchor = header.xpath('./a/text()')
    if len(anchor) > 0:
        labels.append(anchor[0].strip())
    else:
        labels.append(header.text.strip())

print(labels)

['Board Game Rank', '', 'Title', 'Geek Rating', 'Avg Rating', 'Num Voters', 'Shop']


## 5. Web scrapping z wykorzystaniem pakietu BeautifulSoup

In [0]:
url = "https://boardgamegeek.com/"
http = urllib3.PoolManager()
page = http.request("GET", url)

page.data

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


b'\n<!DOCTYPE html>\n<html ng-app="GeekApp" lang="en-US" ng-cloak>\n<head>\n\t<meta charset=\'utf-8\'>\n\t<meta id="vp" name="viewport" content="width=device-width, initial-scale=1.0">\n\t\t<meta content=\'yes\' name=\'apple-mobile-web-app-capable\'>\n\t<meta content=\'IE=edge,chrome=1\' http-equiv=\'X-UA-Compatible\'>\n\n\t\t\t<title>BoardGameGeek | Gaming Unplugged Since 2000</title>\n\t\n\t\n<link rel="apple-touch-icon" \thref="https://cf.geekdo-static.com/icons/touch-icon180.png" />\n<link rel="shortcut icon" \t\thref="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />\n<link rel="icon" \t\t\t\t\thref="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />\n<link rel="search" \t\t\t\thref="/game-opensearch.xml" type="application/opensearchdescription+xml" title="BGG Game Search" />\n<meta name="apple-mobile-web-app-title" content="BGG">\n\n\n\t<meta property="og:image" content="https://cf.geekdo-static.com/images/opengraph/bgg_opengraph.png" />\n\n\n

In [0]:
soup = BeautifulSoup(page.data, 'lxml')

In [0]:
soup

### 5.1 Parsowanie tabeli

In [0]:
url = "https://boardgamegeek.com/browse/boardgame"
http = urllib3.PoolManager()
page = http.request("GET", url)

soup = BeautifulSoup(page.data, 'lxml')

for tr in soup.find('table').find_all('tr'):
    for td in tr.find_all('td'):
      
        if td.find('a'):
            print(td.find('a').text.strip(), end=' ')

            if td.find('span'):
                print(td.find('span').text.strip(), end=' ')
        else:
            print(td.text.strip(), end=' ')
    print()

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



  Gloomhaven (2017) 8.573 8.83 35196 Shop 
  Pandemic Legacy: Season 1 (2015) 8.471 8.62 37186 Shop 
  Terraforming Mars (2016) 8.276 8.43 54287 Shop 
  Brass: Birmingham (2018) 8.275 8.64 12942 Shop 
  Through the Ages: A New Story of Civilization (2015) 8.221 8.47 20107 Shop 
  Twilight Imperium (Fourth Edition) (2017) 8.203 8.69 10709 Shop 
  Star Wars: Rebellion (2016) 8.157 8.42 20320 Shop 
  Twilight Struggle (2005) 8.152 8.30 38091 Shop 
  Gaia Project (2017) 8.151 8.50 13502 Shop 
  Great Western Trail (2016) 8.103 8.29 25192 Shop 
  Scythe (2016) 8.095 8.25 50571 Shop 
  War of the Ring: Second Edition (2012) 8.076 8.46 12008 Shop 
  Spirit Island (2017) 8.049 8.32 19137 Shop 
  The Castles of Burgundy (2011) 8.012 8.13 42414 Shop 
  Terra Mystica (2012) 8.010 8.16 36509 Shop 
  7 Wonders Duel (2015) 7.976 8.10 51393 Shop 
  Concordia (2013) 7.952 8.11 24760 Shop 
  Brass: Lancashire (2007) 7.944 8.15 17608 Shop 
  The 7th Continent (2017) 7.927 8.29 14274 Shop 
  Arkham Horr